<a href="https://colab.research.google.com/github/jechavarria12/World_GDP_Neural_netowrk/blob/main/world_bank_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
#Instalación de las librerías
!pip install -q pandas
!pip install -q numpy
!pip install -q scipy
!pip install -q matplotlib
!pip install -q seaborn
!pip install -q plotly
!pip install -q yellowbrick
!pip install -q scikit-learn
!pip install -q imbalanced-learn
!pip install -q tqdm
!pip install -q joblib
!pip install -q huggingface_hub
!pip install -q datasets
!pip install -q ydata_profiling

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [16]:
#Importar el dataset desde Kaggle
!kaggle datasets download sazidthe1/world-gdp-data
DATASET_NAME = 'world-gdp-data'

%cd {ROOT_DIR}
!unzip -o {DATASET_NAME}.zip -d {DATASET_NAME}

Dataset URL: https://www.kaggle.com/datasets/sazidthe1/world-gdp-data
License(s): Attribution 4.0 International (CC BY 4.0)
100% 108k/108k [00:00<00:00, 537kB/s]
100% 108k/108k [00:00<00:00, 536kB/s]
/
Archive:  world-gdp-data.zip
  inflating: world-gdp-data/country_codes.csv  
  inflating: world-gdp-data/gdp_data.csv  


In [26]:
I#Usar pandas para tener los dos dataframes
import pandas as pd

df_1=pd.read_csv("world-gdp-data//country_codes.csv")
df_2=pd.read_csv('world-gdp-data//gdp_data.csv')

In [31]:
#Unir los Dataframes sobre la columna en común, country code
merged_df = pd.merge(df_1, df_2, on='country_code', how='inner')
print(merged_df.describe())
print(merged_df.isna().sum())
merged_df.dropna(inplace=True)

               year         value
count  10548.000000  1.054800e+04
mean    1994.977436  1.902435e+11
std       17.364673  1.031776e+12
min     1960.000000  8.824746e+06
25%     1981.000000  1.547855e+09
50%     1997.000000  8.140967e+09
75%     2010.000000  5.409850e+10
max     2022.000000  2.550000e+13
country_code    0
region          0
income_group    0
country_name    0
year            0
value           0
dtype: int64


In [43]:
# Pivot the dataframe
pivoted_df = merged_df.pivot_table(index=['country_code', 'region', 'income_group', 'country_name'],
                                   columns='year',
                                   values='value',
                                   aggfunc='first').reset_index()

# Display the pivoted dataframe
display(pivoted_df)

year,country_code,region,income_group,country_name,1960,1961,1962,1963,1964,1965,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,ABW,Latin America & Caribbean,High income,Aruba,NaN,NaN,NaN,NaN,NaN,NaN,...,2.727933e+09,2.791061e+09,2.963128e+09,2.983799e+09,3.092179e+09,3.276188e+09,3.395794e+09,2.610039e+09,3.126019e+09,NaN
1,AFG,South Asia,Low income,Afghanistan,5.377778e+08,5.488889e+08,5.466667e+08,7.511112e+08,8.000000e+08,1.006667e+09,...,2.056449e+10,2.055058e+10,1.999814e+10,1.801955e+10,1.889635e+10,1.841886e+10,1.890450e+10,2.014345e+10,1.458314e+10,NaN
2,AGO,Sub-Saharan Africa,Lower middle income,Angola,NaN,NaN,NaN,NaN,NaN,NaN,...,1.330000e+11,1.370000e+11,8.721930e+10,4.984049e+10,6.897277e+10,7.779294e+10,6.930911e+10,5.024137e+10,6.568544e+10,1.070000e+11
3,ALB,Europe & Central Asia,Upper middle income,Albania,NaN,NaN,NaN,NaN,NaN,NaN,...,1.277622e+10,1.322815e+10,1.138685e+10,1.186120e+10,1.301973e+10,1.515642e+10,1.540183e+10,1.516273e+10,1.793057e+10,1.888210e+10
4,AND,Europe & Central Asia,High income,Andorra,NaN,NaN,NaN,NaN,NaN,NaN,...,3.193513e+09,3.271686e+09,2.789881e+09,2.896610e+09,3.000162e+09,3.218420e+09,3.155149e+09,2.891001e+09,3.325145e+09,3.352033e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,XKX,Europe & Central Asia,Upper middle income,Kosovo,NaN,NaN,NaN,NaN,NaN,NaN,...,6.735328e+09,7.074393e+09,6.295845e+09,6.682674e+09,7.180769e+09,7.878763e+09,7.899741e+09,7.717143e+09,9.412034e+09,9.429156e+09
210,YEM,Middle East & North Africa,Low income,"Yemen, Rep.",NaN,NaN,NaN,NaN,NaN,NaN,...,4.041523e+10,4.322859e+10,4.244449e+10,3.131782e+10,2.684223e+10,2.160616e+10,NaN,NaN,NaN,NaN
211,ZAF,Sub-Saharan Africa,Upper middle income,South Africa,8.748597e+09,9.225996e+09,9.813996e+09,1.085420e+10,1.195600e+10,1.306899e+10,...,4.010000e+11,3.810000e+11,3.470000e+11,3.240000e+11,3.810000e+11,4.040000e+11,3.890000e+11,3.380000e+11,4.190000e+11,4.060000e+11
212,ZMB,Sub-Saharan Africa,Lower middle income,Zambia,7.130000e+08,6.962857e+08,6.931429e+08,7.187143e+08,8.394286e+08,1.082857e+09,...,2.803724e+10,2.714102e+10,2.125122e+10,2.095841e+10,2.587360e+10,2.631151e+10,2.330867e+10,1.811064e+10,2.214765e+10,2.978445e+10


In [45]:
#Se rellena los valores no encontrados con -1
pivoted_df=pivoted_df.fillna(-1)
# Display the pivoted dataframe
display(pivoted_df)

year,country_code,region,income_group,country_name,1960,1961,1962,1963,1964,1965,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,ABW,Latin America & Caribbean,High income,Aruba,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,...,2.727933e+09,2.791061e+09,2.963128e+09,2.983799e+09,3.092179e+09,3.276188e+09,3.395794e+09,2.610039e+09,3.126019e+09,-1.000000e+00
1,AFG,South Asia,Low income,Afghanistan,5.377778e+08,5.488889e+08,5.466667e+08,7.511112e+08,8.000000e+08,1.006667e+09,...,2.056449e+10,2.055058e+10,1.999814e+10,1.801955e+10,1.889635e+10,1.841886e+10,1.890450e+10,2.014345e+10,1.458314e+10,-1.000000e+00
2,AGO,Sub-Saharan Africa,Lower middle income,Angola,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,...,1.330000e+11,1.370000e+11,8.721930e+10,4.984049e+10,6.897277e+10,7.779294e+10,6.930911e+10,5.024137e+10,6.568544e+10,1.070000e+11
3,ALB,Europe & Central Asia,Upper middle income,Albania,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,...,1.277622e+10,1.322815e+10,1.138685e+10,1.186120e+10,1.301973e+10,1.515642e+10,1.540183e+10,1.516273e+10,1.793057e+10,1.888210e+10
4,AND,Europe & Central Asia,High income,Andorra,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,...,3.193513e+09,3.271686e+09,2.789881e+09,2.896610e+09,3.000162e+09,3.218420e+09,3.155149e+09,2.891001e+09,3.325145e+09,3.352033e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,XKX,Europe & Central Asia,Upper middle income,Kosovo,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,...,6.735328e+09,7.074393e+09,6.295845e+09,6.682674e+09,7.180769e+09,7.878763e+09,7.899741e+09,7.717143e+09,9.412034e+09,9.429156e+09
210,YEM,Middle East & North Africa,Low income,"Yemen, Rep.",-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,...,4.041523e+10,4.322859e+10,4.244449e+10,3.131782e+10,2.684223e+10,2.160616e+10,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00
211,ZAF,Sub-Saharan Africa,Upper middle income,South Africa,8.748597e+09,9.225996e+09,9.813996e+09,1.085420e+10,1.195600e+10,1.306899e+10,...,4.010000e+11,3.810000e+11,3.470000e+11,3.240000e+11,3.810000e+11,4.040000e+11,3.890000e+11,3.380000e+11,4.190000e+11,4.060000e+11
212,ZMB,Sub-Saharan Africa,Lower middle income,Zambia,7.130000e+08,6.962857e+08,6.931429e+08,7.187143e+08,8.394286e+08,1.082857e+09,...,2.803724e+10,2.714102e+10,2.125122e+10,2.095841e+10,2.587360e+10,2.631151e+10,2.330867e+10,1.811064e+10,2.214765e+10,2.978445e+10


In [25]:
from ydata_profiling import ProfileReport
display(merged_df.head())

,country_code,region,income_group,country_name,year,value
0,ABW,Latin America & Caribbean,High income,Aruba,1986,405586592.2
1,ABW,Latin America & Caribbean,High income,Aruba,1987,487709497.2
2,ABW,Latin America & Caribbean,High income,Aruba,1988,596648044.7
3,ABW,Latin America & Caribbean,High income,Aruba,1989,695530726.3
4,ABW,Latin America & Caribbean,High income,Aruba,1990,764804469.3
